## Setup

In [1]:
from behavior_base import *
from behavioral_pipeline import GoNogoBehaviorMat
import h5py
import numpy as np
import os

In [2]:
animal = 'JUV011'
session = '211215'

In [3]:
# Get file from server
input_folder = fr"\\filenest.diskstation.me\Wilbrecht_file_server\Madeline\processed_data\{animal}\{session}"
input_file = fr"{animal}_{session}_behaviorLOG.mat"
hfile = h5py.File(os.path.join(input_folder, input_file), 'r')

In [4]:
hfile['out'].keys()

<KeysViewHDF5 ['GoNG_EventTimes', 'directdelivery', 'frame_time', 'portside', 'result', 'run_speed', 'schedule', 'sound_SPL', 'sound_dur', 'sound_freq', 'sound_name']>

## Initialize Processing

In [5]:
eventlist = EventNode(None, None, None, None)
code_map = GoNogoBehaviorMat.code_map
code_map

{3: ('in', 'in'),
 4: ('out', 'out'),
 44: ('out', 'out'),
 81.01: ('outcome', 'no-go_correct_unrewarded'),
 81.02: ('outcome', 'go_correct_unrewarded'),
 81.12: ('outcome', 'go_correct_reward1'),
 81.22: ('outcome', 'go_correct_reward2'),
 82.02: ('outcome', 'no-go_incorrect'),
 83: ('outcome', 'missed'),
 84: ('outcome', 'abort'),
 9.01: ('water_valve', '1'),
 9.02: ('water_valve', '2'),
 9.03: ('water_valve', '3'),
 7.01: ('sound_on', '1'),
 7.02: ('sound_on', '2'),
 7.03: ('sound_on', '3'),
 7.04: ('sound_on', '4'),
 7.05: ('sound_on', '5'),
 7.06: ('sound_on', '6'),
 7.07: ('sound_on', '7'),
 7.08: ('sound_on', '8'),
 7.09: ('sound_on', '9'),
 7.1: ('sound_on', '10'),
 7.11: ('sound_on', '11'),
 7.12: ('sound_on', '12'),
 7.13: ('sound_on', '13'),
 7.14: ('sound_on', '14'),
 7.15: ('sound_on', '15'),
 7.16: ('sound_on', '16')}

In [6]:
trial_events = np.array(hfile['out/GoNG_EventTimes'])
trial_events

array([[3.00000000e+00, 1.51131680e+03, 5.00000000e-01],
       [4.40000000e+01, 1.51134530e+03, 5.00000000e-01],
       [7.01000000e+00, 1.51334647e+03, 1.00000000e+00],
       ...,
       [8.10100000e+01, 5.09153315e+03, 3.88000000e+02],
       [7.16000000e+00, 5.09564763e+03, 3.89000000e+02],
       [8.30000000e+01, 5.09864995e+03, 3.89000000e+02]])

## Processing

In [7]:
for i in range(len(trial_events)):
    eventID, eventTime, trial = trial_events[i]
    eventlist.append(EventNode(code_map[eventID][0] + '|' + code_map[eventID][1], eventTime, trial, eventID))

In [8]:
eventlist.as_df()

,event,etime,trial,ecode
0,in|in,1511.316804,0.5,3.00
1,out|out,1511.345304,0.5,44.00
2,sound_on|1,1513.346466,1.0,7.01
3,in|in,1513.466129,1.0,3.00
4,water_valve|2,1513.468296,1.0,9.02
...,...,...,...,...
9370,out|out,5086.281508,387.5,44.00
9371,sound_on|5,5088.282830,388.0,7.05
9372,outcome|no-go_correct_unrewarded,5091.533150,388.0,81.01
9373,sound_on|16,5095.647634,389.0,7.16


In [9]:
x = GoNogoBehaviorMat(animal, session, os.path.join(input_folder, input_file));

In [10]:
x.to_df()

,animal,session,trial,onset,first_lick_in,last_lick_out,water_valve_on,outcome,sound_num,reward,go_nogo,licks_out,quality,water_valve_amt
0,JUV011,211215,1,1513.346466,1511.316804,1511.345304,1513.468296,1513.468296,1,2,go,1,normal,2
1,JUV011,211215,2,1522.693273,1513.541629,1520.692100,NaN,1522.708592,8,-1,nogo,23,normal,NaN
2,JUV011,211215,3,1536.585370,NaN,1522.715269,NaN,1539.835691,7,0,nogo,1,normal,NaN
3,JUV011,211215,4,1544.402505,1546.756163,NaN,NaN,1546.756163,6,-1,nogo,0,normal,NaN
4,JUV011,211215,5,1565.439921,1546.962495,1563.438596,NaN,1565.691754,6,-1,nogo,28,normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,JUV011,211215,385,5067.524413,5061.328604,5065.523254,NaN,5070.774733,6,0,nogo,11,normal,NaN
385,JUV011,211215,386,5074.868550,NaN,NaN,NaN,5078.118715,5,0,nogo,0,normal,NaN
386,JUV011,211215,387,5081.849034,5082.124188,NaN,5082.126354,5082.126354,2,2,go,0,normal,2
387,JUV011,211215,388,5088.282830,5082.247854,5086.281508,NaN,5091.533150,5,0,nogo,14,normal,NaN


In [11]:
# save as csv file
x.to_df().to_csv(f"{animal}_{session}_behavior_output.csv")